# Inserting data into Notebook
I used 'train.csv' file where i have my dataset for training

In [4]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-9f477a12-1e4d-4b90-ad46-d2d81c68710e',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': 'QoPFoBkCOcy5_z9sCxxIlro_taDDVczipnPs_wXceqeF'
}

configuration_name = 'os_3c4c594765474e10870775abb04cbf2f_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('train.csv', 'defaultikram-donotdelete-pr-ht8dbkrt3n4iv5'))
df.take(5)


[Row(battery_power='842', blue='0', clock_speed='2.2', dual_sim='0', fc='1', four_g='0', int_memory='7', m_dep='0.6', mobile_wt='188', n_cores='2', pc='2', px_height='20', px_width='756', ram='2549', sc_h='9', sc_w='7', talk_time='19', three_g='0', touch_screen='0', wifi='1', price_range='1'),
 Row(battery_power='1021', blue='1', clock_speed='0.5', dual_sim='1', fc='0', four_g='1', int_memory='53', m_dep='0.7', mobile_wt='136', n_cores='3', pc='6', px_height='905', px_width='1988', ram='2631', sc_h='17', sc_w='3', talk_time='7', three_g='1', touch_screen='1', wifi='0', price_range='2'),
 Row(battery_power='563', blue='1', clock_speed='0.5', dual_sim='1', fc='2', four_g='1', int_memory='41', m_dep='0.9', mobile_wt='145', n_cores='5', pc='6', px_height='1263', px_width='1716', ram='2603', sc_h='11', sc_w='2', talk_time='9', three_g='1', touch_screen='1', wifi='0', price_range='2'),
 Row(battery_power='615', blue='1', clock_speed='2.5', dual_sim='0', fc='0', four_g='0', int_memory='10', m

In [6]:
df.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [7]:
df.createOrReplaceTempView('Dataset')

# Initial Data Exploration
Now, I am going to implement some simple functions that i learned earlier so that i can have a view of my dataset. I will find min, mix, standard deviation and some other properties of my data columns and will explore them on different columns. I have imported data using pandas dataframe but now converted it to spark and created a view so that i can run quries on it.

In [8]:
"""
    function to find minimum of a column
"""
def findMin(columnName):
    spark.sql('select min('+columName+') from Dataset')

In [ ]:
def findMax(columnName):
    spark.sql('select min('+columName+') from Dataset')